In [4]:
import os
import datetime
from torchkeras import Model, summary

# 打印时间
def printbar():
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n"+"==========" * 8 + "%s" % nowtime)

# mac系统上pytorch和matplotlib在jupyter中同时跑需要更改环境变量
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" 

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


OSError: [WinError 126] 找不到指定的模块。 Error loading "C:\Users\z\Anaconda3\lib\site-packages\torch\lib\asmjit.dll" or one of its dependencies.

In [3]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt 
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
%matplotlib inline

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


OSError: [WinError 126] 找不到指定的模块。 Error loading "C:\Users\z\Anaconda3\lib\site-packages\torch\lib\asmjit.dll" or one of its dependencies.

## 线性回归模型

### 准备数据